In [3]:
import cv2
import numpy as np
from PIL import Image
import os
import inspect
from skimage.metrics import structural_similarity as ssim
from pathlib import Path
from utils._common import _to_uint8_rgb
from utils import modelLoader

# Table of Contents

* [NewtonFool](#NewtonFool)
* [Projected Gradient Descent](#Projected-Gradient-Descent)
* [Basic Iterative Method](#Basic-Iterative-Method)
* [Hop Skip Jump Attack](#Hop-Skip-Jump-Attack)
* [Geometric Decision Based Attack](#Geometric-Decision-Based-Attack)
* [GRAPHITE Whitebox](#GRAPHITE-Whitebox)
* [Fast Gradient Method](#Fast-Gradient-Method)

# NewtonFool

In [2]:
from NewtonFool_attack import NewtonFool_attack

folder = Path("../to_attack")
image_exts = {".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff", ".webp"}
image_paths = sorted(
    str(p) for p in folder.iterdir()
    if p.is_file() and p.suffix.lower() in image_exts
)
print(f"Found {len(image_paths)} images")

resnet50_Model = modelLoader.loadModel("resnet50", "utils/.models/resnet50.pth")
# densenet121_Model = modelLoader.loadModel("densenet121", ".models/densenet121.pth")

allSSIMScores = []
successfulAttacks = 0

# ==== configure output folder ====
adv_output_dir = Path("../adv_images")
adv_output_dir.mkdir(parents=True, exist_ok=True)

'''
def _finalize(orig_uint8: np.ndarray, x_adv01: np.ndarray) -> tuple:
    adv_uint8 = _to_uint8_rgb(x_adv01[0])  # (H_adv, W_adv, 3)

    # Resize original to match adv size if needed
    H_adv, W_adv = adv_uint8.shape[:2]
    if orig_uint8.shape[:2] != (H_adv, W_adv):
        import cv2
        orig_match = cv2.resize(orig_uint8, (W_adv, H_adv), interpolation=cv2.INTER_LINEAR)
    else:
        orig_match = orig_uint8

    noise = adv_uint8.astype(np.int16) - orig_match.astype(np.int16)
    return orig_match, noise, adv_uint8
'''

for p in image_paths:
    print("Processing:", p)
    try:
        ##################################### ATTACK METHOD ########################################
        
        orig_u8, adv = NewtonFool_attack(resnet50_Model, p)

        ############################################################################################

        adv = np.transpose(adv, (1, 2, 0))
        _adv = adv
        #print(orig_u8.shape)
        #print(adv.shape)
        if orig_u8.shape[:2] != adv.shape[:2]:
            _adv = cv2.resize(
                adv, (orig_u8.shape[1], orig_u8.shape[0]),
                interpolation=cv2.INTER_LINEAR
            )
        adv_u8 = _to_uint8_rgb(_adv)
        noise = adv_u8.astype(np.int16) - orig_u8.astype(np.int16)
    
        # Save adversarial image to output folder
        out_path = adv_output_dir / Path(p).name
        cv2.imwrite(str(out_path), cv2.cvtColor(adv_u8, cv2.COLOR_RGB2BGR))
    
        # Compute SSIM
        ssim_kwargs = {"data_range": 255}
        if "channel_axis" in inspect.signature(ssim).parameters:
            ssim_kwargs["channel_axis"] = -1
        else:
            ssim_kwargs["multichannel"] = True
    
        #print(ssim_kwargs)
        ssim_score = ssim(orig_u8, adv_u8, **ssim_kwargs)
        allSSIMScores.append(ssim_score)
        print(f"SSIM between original and adversarial: {ssim_score:.6f}")
        print(f"Saved adversarial image: {out_path}")
        print(" ")
        successfulAttacks += 1
        
    except Exception as e:
        print("Error processing:", p)
        print(e)
        continue


print("")
print("Average SSIM Score:", np.mean(allSSIMScores))
print("Successful Attacks:", successfulAttacks)

Found 2 images
[MODEL] Loading 'resnet50' weights from utils/.models/resnet50.pth ...
[MODEL] 'resnet50' ready on cuda:0
Processing: ../to_attack/img_0337.png


NewtonFool:   0%|          | 0/1 [00:00<?, ?it/s]

SSIM between original and adversarial: 0.999997
Saved adversarial image: ../adv_images/img_0337.png
 
Processing: ../to_attack/img_0667.png


NewtonFool:   0%|          | 0/1 [00:00<?, ?it/s]

SSIM between original and adversarial: 1.000000
Saved adversarial image: ../adv_images/img_0667.png
 

Average SSIM Score: 0.9999984560913506
Successful Attacks: 2


# Projected Gradient Descent

In [2]:
from PGD_attack import PGD_attack

folder = Path("../to_attack")
image_exts = {".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff", ".webp"}
image_paths = sorted(
    str(p) for p in folder.iterdir()
    if p.is_file() and p.suffix.lower() in image_exts
)
print(f"Found {len(image_paths)} images")

resnet50_Model = modelLoader.loadModel("resnet50", "utils/.models/resnet50.pth")
# densenet121_Model = modelLoader.loadModel("densenet121", ".models/densenet121.pth")

allSSIMScores = []
successfulAttacks = 0

# ==== configure output folder ====
adv_output_dir = Path("../adv_images")
adv_output_dir.mkdir(parents=True, exist_ok=True)

'''
def _finalize(orig_uint8: np.ndarray, x_adv01: np.ndarray) -> tuple:
    adv_uint8 = _to_uint8_rgb(x_adv01[0])  # (H_adv, W_adv, 3)

    # Resize original to match adv size if needed
    H_adv, W_adv = adv_uint8.shape[:2]
    if orig_uint8.shape[:2] != (H_adv, W_adv):
        import cv2
        orig_match = cv2.resize(orig_uint8, (W_adv, H_adv), interpolation=cv2.INTER_LINEAR)
    else:
        orig_match = orig_uint8

    noise = adv_uint8.astype(np.int16) - orig_match.astype(np.int16)
    return orig_match, noise, adv_uint8
'''

for p in image_paths:
    print("Processing:", p)
    try:
        ##################################### ATTACK METHOD ########################################
        
        orig_u8, adv = PGD_attack(resnet50_Model, p)

        ############################################################################################

        adv = np.transpose(adv, (1, 2, 0))
        _adv = adv
        #print(orig_u8.shape)
        #print(adv.shape)
        if orig_u8.shape[:2] != adv.shape[:2]:
            _adv = cv2.resize(
                adv, (orig_u8.shape[1], orig_u8.shape[0]),
                interpolation=cv2.INTER_LINEAR
            )
        adv_u8 = _to_uint8_rgb(_adv)
        noise = adv_u8.astype(np.int16) - orig_u8.astype(np.int16)
    
        # Save adversarial image to output folder
        out_path = adv_output_dir / Path(p).name
        cv2.imwrite(str(out_path), cv2.cvtColor(adv_u8, cv2.COLOR_RGB2BGR))
    
        # Compute SSIM
        ssim_kwargs = {"data_range": 255}
        if "channel_axis" in inspect.signature(ssim).parameters:
            ssim_kwargs["channel_axis"] = -1
        else:
            ssim_kwargs["multichannel"] = True
    
        #print(ssim_kwargs)
        ssim_score = ssim(orig_u8, adv_u8, **ssim_kwargs)
        allSSIMScores.append(ssim_score)
        print(f"SSIM between original and adversarial: {ssim_score:.6f}")
        print(f"Saved adversarial image: {out_path}")
        print(" ")
        successfulAttacks += 1
        
    except Exception as e:
        print("Error processing:", p)
        print(e)
        continue


print("")
print("Average SSIM Score:", np.mean(allSSIMScores))
print("Successful Attacks:", successfulAttacks)

Found 2 images
[MODEL] Loading 'resnet50' weights from utils/.models/resnet50.pth ...
[MODEL] 'resnet50' ready on cuda:0
Processing: ../to_attack/img_0337.png


PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

SSIM between original and adversarial: 0.125584
Saved adversarial image: ../adv_images/img_0337.png
 
Processing: ../to_attack/img_0667.png


PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

SSIM between original and adversarial: 0.146463
Saved adversarial image: ../adv_images/img_0667.png
 

Average SSIM Score: 0.13602344228343324
Successful Attacks: 2


# Basic Iterative Method

In [3]:
from BIM_attack import BIM_attack

folder = Path("../to_attack")
image_exts = {".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff", ".webp"}
image_paths = sorted(
    str(p) for p in folder.iterdir()
    if p.is_file() and p.suffix.lower() in image_exts
)
print(f"Found {len(image_paths)} images")

resnet50_Model = modelLoader.loadModel("resnet50", "utils/.models/resnet50.pth")
# densenet121_Model = modelLoader.loadModel("densenet121", ".models/densenet121.pth")

allSSIMScores = []
successfulAttacks = 0

# ==== configure output folder ====
adv_output_dir = Path("../adv_images")
adv_output_dir.mkdir(parents=True, exist_ok=True)

'''
def _finalize(orig_uint8: np.ndarray, x_adv01: np.ndarray) -> tuple:
    adv_uint8 = _to_uint8_rgb(x_adv01[0])  # (H_adv, W_adv, 3)

    # Resize original to match adv size if needed
    H_adv, W_adv = adv_uint8.shape[:2]
    if orig_uint8.shape[:2] != (H_adv, W_adv):
        import cv2
        orig_match = cv2.resize(orig_uint8, (W_adv, H_adv), interpolation=cv2.INTER_LINEAR)
    else:
        orig_match = orig_uint8

    noise = adv_uint8.astype(np.int16) - orig_match.astype(np.int16)
    return orig_match, noise, adv_uint8
'''

for p in image_paths:
    print("Processing:", p)
    try:
        ##################################### ATTACK METHOD ########################################
        
        orig_u8, adv = BIM_attack(resnet50_Model, p)

        ############################################################################################

        adv = np.transpose(adv, (1, 2, 0))
        _adv = adv
        #print(orig_u8.shape)
        #print(adv.shape)
        if orig_u8.shape[:2] != adv.shape[:2]:
            _adv = cv2.resize(
                adv, (orig_u8.shape[1], orig_u8.shape[0]),
                interpolation=cv2.INTER_LINEAR
            )
        adv_u8 = _to_uint8_rgb(_adv)
        noise = adv_u8.astype(np.int16) - orig_u8.astype(np.int16)
    
        # Save adversarial image to output folder
        out_path = adv_output_dir / Path(p).name
        cv2.imwrite(str(out_path), cv2.cvtColor(adv_u8, cv2.COLOR_RGB2BGR))
    
        # Compute SSIM
        ssim_kwargs = {"data_range": 255}
        if "channel_axis" in inspect.signature(ssim).parameters:
            ssim_kwargs["channel_axis"] = -1
        else:
            ssim_kwargs["multichannel"] = True
    
        #print(ssim_kwargs)
        ssim_score = ssim(orig_u8, adv_u8, **ssim_kwargs)
        allSSIMScores.append(ssim_score)
        print(f"SSIM between original and adversarial: {ssim_score:.6f}")
        print(f"Saved adversarial image: {out_path}")
        print(" ")
        successfulAttacks += 1
        
    except Exception as e:
        print("Error processing:", p)
        print(e)
        continue


print("")
print("Average SSIM Score:", np.mean(allSSIMScores))
print("Successful Attacks:", successfulAttacks)

Found 2 images
[MODEL] Loading 'resnet50' weights from utils/.models/resnet50.pth ...
[MODEL] 'resnet50' ready on cuda:0
Processing: ../to_attack/img_0337.png


PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

SSIM between original and adversarial: 0.125584
Saved adversarial image: ../adv_images/img_0337.png
 
Processing: ../to_attack/img_0667.png


PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

SSIM between original and adversarial: 0.146463
Saved adversarial image: ../adv_images/img_0667.png
 

Average SSIM Score: 0.13602344228343324
Successful Attacks: 2


# Hop Skip Jump Attack

In [3]:
from HopSkipJump_attack import HopSkipJump_attack

folder = Path("../to_attack")
image_exts = {".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff", ".webp"}
image_paths = sorted(
    str(p) for p in folder.iterdir()
    if p.is_file() and p.suffix.lower() in image_exts
)
print(f"Found {len(image_paths)} images")

resnet50_Model = modelLoader.loadModel("resnet50", "utils/.models/resnet50.pth")
# densenet121_Model = modelLoader.loadModel("densenet121", ".models/densenet121.pth")

allSSIMScores = []
successfulAttacks = 0

# ==== configure output folder ====
adv_output_dir = Path("../adv_images")
adv_output_dir.mkdir(parents=True, exist_ok=True)

'''
def _finalize(orig_uint8: np.ndarray, x_adv01: np.ndarray) -> tuple:
    adv_uint8 = _to_uint8_rgb(x_adv01[0])  # (H_adv, W_adv, 3)

    # Resize original to match adv size if needed
    H_adv, W_adv = adv_uint8.shape[:2]
    if orig_uint8.shape[:2] != (H_adv, W_adv):
        import cv2
        orig_match = cv2.resize(orig_uint8, (W_adv, H_adv), interpolation=cv2.INTER_LINEAR)
    else:
        orig_match = orig_uint8

    noise = adv_uint8.astype(np.int16) - orig_match.astype(np.int16)
    return orig_match, noise, adv_uint8
'''

for p in image_paths:
    print("Processing:", p)
    try:
        ##################################### ATTACK METHOD ########################################
        
        orig_u8, adv = HopSkipJump_attack(resnet50_Model, p)

        ############################################################################################

        adv = np.transpose(adv, (1, 2, 0))
        _adv = adv
        #print(orig_u8.shape)
        #print(adv.shape)
        if orig_u8.shape[:2] != adv.shape[:2]:
            _adv = cv2.resize(
                adv, (orig_u8.shape[1], orig_u8.shape[0]),
                interpolation=cv2.INTER_LINEAR
            )
        adv_u8 = _to_uint8_rgb(_adv)
        noise = adv_u8.astype(np.int16) - orig_u8.astype(np.int16)
    
        # Save adversarial image to output folder
        out_path = adv_output_dir / Path(p).name
        cv2.imwrite(str(out_path), cv2.cvtColor(adv_u8, cv2.COLOR_RGB2BGR))
    
        # Compute SSIM
        ssim_kwargs = {"data_range": 255}
        if "channel_axis" in inspect.signature(ssim).parameters:
            ssim_kwargs["channel_axis"] = -1
        else:
            ssim_kwargs["multichannel"] = True
    
        #print(ssim_kwargs)
        ssim_score = ssim(orig_u8, adv_u8, **ssim_kwargs)
        allSSIMScores.append(ssim_score)
        print(f"SSIM between original and adversarial: {ssim_score:.6f}")
        print(f"Saved adversarial image: {out_path}")
        print(" ")
        successfulAttacks += 1
        
    except Exception as e:
        print("Error processing:", p)
        print(e)
        continue


print("")
print("Average SSIM Score:", np.mean(allSSIMScores))
print("Successful Attacks:", successfulAttacks)

Found 2 images
[MODEL] Loading 'resnet50' weights from utils/.models/resnet50.pth ...
[MODEL] 'resnet50' ready on cuda:0
Processing: ../to_attack/img_0337.png


HopSkipJump:   0%|          | 0/1 [00:00<?, ?it/s]

SSIM between original and adversarial: 1.000000
Saved adversarial image: ../adv_images/img_0337.png
 
Processing: ../to_attack/img_0667.png


HopSkipJump:   0%|          | 0/1 [00:00<?, ?it/s]

SSIM between original and adversarial: 0.999997
Saved adversarial image: ../adv_images/img_0667.png
 

Average SSIM Score: 0.9999985351031144
Successful Attacks: 2


# Geometric Decision Based Attack

In [2]:
from GeoD_attack import GeoD_attack

folder = Path("../to_attack")
image_exts = {".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff", ".webp"}
image_paths = sorted(
    str(p) for p in folder.iterdir()
    if p.is_file() and p.suffix.lower() in image_exts
)
print(f"Found {len(image_paths)} images")

resnet50_Model = modelLoader.loadModel("resnet50", "utils/.models/resnet50.pth")
# densenet121_Model = modelLoader.loadModel("densenet121", ".models/densenet121.pth")

allSSIMScores = []
successfulAttacks = 0

# ==== configure output folder ====
adv_output_dir = Path("../adv_images")
adv_output_dir.mkdir(parents=True, exist_ok=True)

'''
def _finalize(orig_uint8: np.ndarray, x_adv01: np.ndarray) -> tuple:
    adv_uint8 = _to_uint8_rgb(x_adv01[0])  # (H_adv, W_adv, 3)

    # Resize original to match adv size if needed
    H_adv, W_adv = adv_uint8.shape[:2]
    if orig_uint8.shape[:2] != (H_adv, W_adv):
        import cv2
        orig_match = cv2.resize(orig_uint8, (W_adv, H_adv), interpolation=cv2.INTER_LINEAR)
    else:
        orig_match = orig_uint8

    noise = adv_uint8.astype(np.int16) - orig_match.astype(np.int16)
    return orig_match, noise, adv_uint8
'''

for p in image_paths:
    print("Processing:", p)
    try:
        ##################################### ATTACK METHOD ########################################
        
        orig_u8, adv = GeoD_attack(resnet50_Model, p, norm=np.inf)

        ############################################################################################

        adv = np.transpose(adv, (1, 2, 0))
        _adv = adv
        print(orig_u8.shape)
        print(adv.shape)
        if orig_u8.shape[:2] != adv.shape[:2]:
            _adv = cv2.resize(
                adv, (orig_u8.shape[1], orig_u8.shape[0]),
                interpolation=cv2.INTER_LINEAR
            )
        adv_u8 = _to_uint8_rgb(_adv)
        noise = adv_u8.astype(np.int16) - orig_u8.astype(np.int16)
    
        # Save adversarial image to output folder
        out_path = adv_output_dir / Path(p).name
        cv2.imwrite(str(out_path), cv2.cvtColor(adv_u8, cv2.COLOR_RGB2BGR))
    
        # Compute SSIM
        ssim_kwargs = {"data_range": 255}
        if "channel_axis" in inspect.signature(ssim).parameters:
            ssim_kwargs["channel_axis"] = -1
        else:
            ssim_kwargs["multichannel"] = True
    
        #print(ssim_kwargs)
        ssim_score = ssim(orig_u8, adv_u8, **ssim_kwargs)
        allSSIMScores.append(ssim_score)
        print(f"SSIM between original and adversarial: {ssim_score:.6f}")
        print(f"Saved adversarial image: {out_path}")
        print(" ")
        successfulAttacks += 1
        
    except Exception as e:
        print("Error processing:", p)
        print(e)
        continue


print("")
print("Average SSIM Score:", np.mean(allSSIMScores))
print("Successful Attacks:", successfulAttacks)

Found 2 images
[MODEL] Loading 'resnet50' weights from utils/.models/resnet50.pth ...
[MODEL] 'resnet50' ready on cuda:0
Processing: ../to_attack/img_0337.png


GeoDA - samples:   0%|          | 0/1 [00:00<?, ?it/s]

GeoDA - steps:   0%|          | 0/9 [00:00<?, ?it/s]

(256, 256, 3)
(256, 256, 3)
SSIM between original and adversarial: 0.955711
Saved adversarial image: ../adv_images/img_0337.png
 
Processing: ../to_attack/img_0667.png


GeoDA - samples:   0%|          | 0/1 [00:00<?, ?it/s]

GeoDA - steps:   0%|          | 0/9 [00:00<?, ?it/s]

(256, 256, 3)
(256, 256, 3)
SSIM between original and adversarial: 0.082951
Saved adversarial image: ../adv_images/img_0667.png
 

Average SSIM Score: 0.5193308685643822
Successful Attacks: 2


# GRAPHITE Whitebox

In [ ]:
from GRAPHITEWhitebox_attack import GRAPHITEWhitebox_attack

folder = Path("../to_attack")
image_exts = {".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff", ".webp"}
image_paths = sorted(
    str(p) for p in folder.iterdir()
    if p.is_file() and p.suffix.lower() in image_exts
)
print(f"Found {len(image_paths)} images")

resnet50_Model = modelLoader.loadModel("resnet50", "utils/.models/resnet50.pth")
# densenet121_Model = modelLoader.loadModel("densenet121", ".models/densenet121.pth")

allSSIMScores = []
successfulAttacks = 0

# ==== configure output folder ====
adv_output_dir = Path("../adv_images")
adv_output_dir.mkdir(parents=True, exist_ok=True)

'''
def _finalize(orig_uint8: np.ndarray, x_adv01: np.ndarray) -> tuple:
    adv_uint8 = _to_uint8_rgb(x_adv01[0])  # (H_adv, W_adv, 3)

    # Resize original to match adv size if needed
    H_adv, W_adv = adv_uint8.shape[:2]
    if orig_uint8.shape[:2] != (H_adv, W_adv):
        import cv2
        orig_match = cv2.resize(orig_uint8, (W_adv, H_adv), interpolation=cv2.INTER_LINEAR)
    else:
        orig_match = orig_uint8

    noise = adv_uint8.astype(np.int16) - orig_match.astype(np.int16)
    return orig_match, noise, adv_uint8
'''

for p in image_paths:
    print("Processing:", p)
    try:
        ##################################### ATTACK METHOD ########################################
        
        orig_u8, adv = GRAPHITEWhitebox_attack(resnet50_Model, p)

        ############################################################################################

        adv = np.transpose(adv, (1, 2, 0))
        _adv = adv
        #print(orig_u8.shape)
        #print(adv.shape)
        if orig_u8.shape[:2] != adv.shape[:2]:
            _adv = cv2.resize(
                adv, (orig_u8.shape[1], orig_u8.shape[0]),
                interpolation=cv2.INTER_LINEAR
            )
        adv_u8 = _to_uint8_rgb(_adv)
        noise = adv_u8.astype(np.int16) - orig_u8.astype(np.int16)
    
        # Save adversarial image to output folder
        out_path = adv_output_dir / Path(p).name
        cv2.imwrite(str(out_path), cv2.cvtColor(adv_u8, cv2.COLOR_RGB2BGR))
    
        # Compute SSIM
        ssim_kwargs = {"data_range": 255}
        if "channel_axis" in inspect.signature(ssim).parameters:
            ssim_kwargs["channel_axis"] = -1
        else:
            ssim_kwargs["multichannel"] = True
    
        #print(ssim_kwargs)
        ssim_score = ssim(orig_u8, adv_u8, **ssim_kwargs)
        allSSIMScores.append(ssim_score)
        print(f"SSIM between original and adversarial: {ssim_score:.6f}")
        print(f"Saved adversarial image: {out_path}")
        print(" ")
        successfulAttacks += 1
        
    except Exception as e:
        print("Error processing:", p)
        print(e)
        continue


print("")
print("Average SSIM Score:", np.mean(allSSIMScores))
print("Successful Attacks:", successfulAttacks)

Found 2 images
[MODEL] Loading 'resnet50' weights from utils/.models/resnet50.pth ...
[MODEL] 'resnet50' ready on cuda:0
Processing: ../to_attack/img_0337.png


# Fast Gradient Method

In [2]:
from FGM_attack import FGM_attack

folder = Path("../to_attack")
image_exts = {".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff", ".webp"}
image_paths = sorted(
    str(p) for p in folder.iterdir()
    if p.is_file() and p.suffix.lower() in image_exts
)
print(f"Found {len(image_paths)} images")

resnet50_Model = modelLoader.loadModel("resnet50", "utils/.models/resnet50.pth")
# densenet121_Model = modelLoader.loadModel("densenet121", ".models/densenet121.pth")

allSSIMScores = []
successfulAttacks = 0

# ==== configure output folder ====
adv_output_dir = Path("../adv_images")
adv_output_dir.mkdir(parents=True, exist_ok=True)

'''
def _finalize(orig_uint8: np.ndarray, x_adv01: np.ndarray) -> tuple:
    adv_uint8 = _to_uint8_rgb(x_adv01[0])  # (H_adv, W_adv, 3)

    # Resize original to match adv size if needed
    H_adv, W_adv = adv_uint8.shape[:2]
    if orig_uint8.shape[:2] != (H_adv, W_adv):
        import cv2
        orig_match = cv2.resize(orig_uint8, (W_adv, H_adv), interpolation=cv2.INTER_LINEAR)
    else:
        orig_match = orig_uint8

    noise = adv_uint8.astype(np.int16) - orig_match.astype(np.int16)
    return orig_match, noise, adv_uint8
'''

for p in image_paths:
    print("Processing:", p)
    try:
        ##################################### ATTACK METHOD ########################################
        
        orig_u8, adv = FGM_attack(resnet50_Model, p, norm=np.inf)

        ############################################################################################

        adv = np.transpose(adv, (1, 2, 0))
        _adv = adv
        #print(orig_u8.shape)
        #print(adv.shape)
        if orig_u8.shape[:2] != adv.shape[:2]:
            _adv = cv2.resize(
                adv, (orig_u8.shape[1], orig_u8.shape[0]),
                interpolation=cv2.INTER_LINEAR
            )
        adv_u8 = _to_uint8_rgb(_adv)
        noise = adv_u8.astype(np.int16) - orig_u8.astype(np.int16)
    
        # Save adversarial image to output folder
        out_path = adv_output_dir / Path(p).name
        cv2.imwrite(str(out_path), cv2.cvtColor(adv_u8, cv2.COLOR_RGB2BGR))
    
        # Compute SSIM
        ssim_kwargs = {"data_range": 255}
        if "channel_axis" in inspect.signature(ssim).parameters:
            ssim_kwargs["channel_axis"] = -1
        else:
            ssim_kwargs["multichannel"] = True
    
        #print(ssim_kwargs)
        ssim_score = ssim(orig_u8, adv_u8, **ssim_kwargs)
        allSSIMScores.append(ssim_score)
        print(f"SSIM between original and adversarial: {ssim_score:.6f}")
        print(f"Saved adversarial image: {out_path}")
        print(" ")
        successfulAttacks += 1
        
    except Exception as e:
        print("Error processing:", p)
        print(e)
        continue


print("")
print("Average SSIM Score:", np.mean(allSSIMScores))
print("Successful Attacks:", successfulAttacks)

Found 2 images
[MODEL] Loading 'resnet50' weights from utils/.models/resnet50.pth ...
[MODEL] 'resnet50' ready on cuda:0
Processing: ../to_attack/img_0337.png
SSIM between original and adversarial: 0.065847
Saved adversarial image: ../adv_images/img_0337.png
 
Processing: ../to_attack/img_0667.png
SSIM between original and adversarial: 0.090883
Saved adversarial image: ../adv_images/img_0667.png
 

Average SSIM Score: 0.07836504145556888
Successful Attacks: 2


**==================================== EXAMPLE FROM THE ART GIT REPO ============================================**

Found 2 images
[MODEL] Loading 'resnet50' weights from utils/.models/resnet50.pth ...
[MODEL] 'resnet50' ready on cuda:0
Processing: ../to_attack/img_0337.png


C&W L_inf:   0%|          | 0/1 [00:00<?, ?it/s]

SSIM between original and adversarial: 0.992852
Saved adversarial image: ../adv_images/img_0337.png
 
Processing: ../to_attack/img_0667.png


C&W L_inf:   0%|          | 0/1 [00:00<?, ?it/s]

SSIM between original and adversarial: 0.997221
Saved adversarial image: ../adv_images/img_0667.png
 

Average SSIM Score: 0.9950366083050868
Successful Attacks: 2


In [3]:
"""
The script demonstrates a simple example of using ART with PyTorch. The example train a small model on the MNIST dataset
and creates adversarial examples using the Fast Gradient Sign Method. Here we use the ART classifier to train the model,
it would also be possible to provide a pretrained model to the ART classifier.
The parameters are chosen for reduced computational requirements of the script and not optimised for accuracy.
"""

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import PyTorchClassifier
from art.utils import load_mnist

In [4]:
# Step 0: Define the neural network model, return logits instead of activation in forward method

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=4, kernel_size=5, stride=1)
        self.conv_2 = nn.Conv2d(in_channels=4, out_channels=10, kernel_size=5, stride=1)
        self.fc_1 = nn.Linear(in_features=4 * 4 * 10, out_features=100)
        self.fc_2 = nn.Linear(in_features=100, out_features=10)

    def forward(self, x):
        x = F.relu(self.conv_1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv_2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4 * 4 * 10)
        x = F.relu(self.fc_1(x))
        x = self.fc_2(x)
        return x


# Step 1: Load the MNIST dataset

(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_mnist()

# Step 1a: Swap axes to PyTorch's NCHW format

x_train = np.transpose(x_train, (0, 3, 1, 2)).astype(np.float32)
x_test = np.transpose(x_test, (0, 3, 1, 2)).astype(np.float32)
print(x_train.shape)
print(x_test.shape)

(60000, 1, 28, 28)
(10000, 1, 28, 28)


In [6]:
# Step 2: Create the model

model = Net()

# Step 2a: Define the loss function and the optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Step 3: Create the ART classifier

classifier = PyTorchClassifier(
    model=model,
    clip_values=(min_pixel_value, max_pixel_value),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(1, 28, 28),
    nb_classes=10,
)

# Step 4: Train the ART classifier

classifier.fit(x_train, y_train, batch_size=64, nb_epochs=3)

# Step 5: Evaluate the ART classifier on benign test examples

predictions = classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

# Step 6: Generate adversarial test examples
attack = FastGradientMethod(classifier, eps=0.2)
x_test_adv = attack.generate(x=x_test)

# Step 7: Evaluate the ART classifier on adversarial test examples

predictions = classifier.predict(x_test_adv)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on adversarial test examples: {}%".format(accuracy * 100))

Accuracy on benign test examples: 97.11%
Accuracy on adversarial test examples: 35.589999999999996%
